# Trying to pull "random" tracks from Spotify

# Imports

In [1]:
# from furl import furl
# from math import ceil

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

import json
import requests
import sys

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import pprint
import time

pp = pprint.PrettyPrinter()

%config InlineBackend.figure_format ='retina'

pd.options.display.max_rows = 200
pd.options.display.max_columns = 2000

### Setting Global Variables:

In [2]:
SPOTIFY_CLIENT_ID='MY_CLIENT_ID'
SPOTIFY_CLIENT_SECRET='MY_CLIENT_SECRET'
SPOTIFY_REDIRECT_URI='MY_REDIRECT_URI'

### Getting "Random" Tracks:

In [3]:
tracks = {}

In [4]:
username = 'MY_USERNAME'
scope = 'user-read-private user-read-email user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

### Below code is inspired by / adapted for Python from Perry Janssen's Medium article 
### "Getting random tracks using the Spotify API"
### https://medium.com/@perryjanssen/getting-random-tracks-using-the-spotify-api-61889b0c0c27
### also used this:
### https://stackoverflow.com/questions/2823316/generate-a-random-letter-in-python


characters = 'abcdefghijklmnopqrstuvwxyz'


for searches in range(20):
    if token:
        sp = spotipy.Spotify(auth=token)
        rand_char = random.choice(characters)
        search = rand_char + '%'
        for i in range(40):

            results  = sp.search(q=search,
                                 type='track',
                                 limit=50, 
                                 offset=(i*50))
            for track in results['tracks']['items']:
    #             track = item['id']
                tracks[track['id']] = [track['name'], # track title
                                 track['artists'][0]['name'], # artist name
                                 track['artists'][0]['id'], # artist id
                                 track['album']['name'], # album title
                                 track['album']['id'], # album id
                                 track['album']['release_date'], # date released 
                                 track['popularity']] # popularity of track
    #            track_info = {track['name']: [track['artists'][0]['name'], track['album']['name']]}
    #             all_saved_tracks[track['id']] = [track['name'], # track title
    #                                              track['artists'][0]['name'], # artist name
    #                                              track['artists'][0]['id'], # artist id
    #                                              track['album']['name'], # album title
    #                                              track['album']['id'], # album id
    #                                              item['added_at']] # time added 
    else:
        print("Can't get token for", username)


 


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

### Putting Tracks Into a DataFrame

In [ ]:
df = pd.DataFrame.from_dict(tracks, orient='index', 
                                                  columns=['title', 
                                                           'artist', 
                                                           'artist_id', 
                                                           'album', 
                                                           'album_id', 
                                                           'release_date',
                                                          'popularity']).reset_index()

In [ ]:
df.to_csv('data/search_tracks_info.csv', index=False)

In [ ]:
df.describe()

In [ ]:
df[df.duplicated() == True]

In [ ]:
df.head()

In [ ]:
df[df.duplicated(subset=['artist']) == True]['artist'].value_counts()

In [ ]:
df[df.duplicated(subset=['title']) == True].shape

In [ ]:
df[df.duplicated(subset=['title', 'artist'], keep=False) == True].sort_values('title')

# Getting tracks from genre seeds

In [ ]:
genre_seeds = sp.recommendation_genre_seeds()['genres']

In [ ]:
genre_seeds

In [ ]:
len(genre_seeds)

In [ ]:
genre_tracks = {}

In [ ]:
username = 'MY_USERNAME'
scope = 'user-read-private user-read-email user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)


# for searches in range(1):
for each in genre_seeds:
    genre = each
    if token:
        sp = spotipy.Spotify(auth=token)
        for i in range(10):
            results  = sp.recommendations(seed_genres=[each],
                                 limit=100)
            for track in results['tracks']:
                id = track['id']
                genre_tracks[id] = [track['name'], # track title
                                    track['artists'][0]['name'], # artist name
                                    track['artists'][0]['id'], # artist id
                                    track['album']['name'], # album title
                                    track['album']['id'], # album id
                                    track['album']['release_date'], # date released 
                                    track['popularity'], # popularity of track
                                    genre] # seed used
    #            track_info = {track['name']: [track['artists'][0]['name'], track['album']['name']]}
    #             all_saved_tracks[track['id']] = [track['name'], # track title
    #                                              track['artists'][0]['name'], # artist name
    #                                              track['artists'][0]['id'], # artist id
    #                                              track['album']['name'], # album title
    #                                              track['album']['id'], # album id
    #                                              item['added_at']] # time added 
    else:
        print("Can't get token for", username)

In [ ]:
results

In [ ]:
genre_tracks_df = pd.DataFrame.from_dict(genre_tracks, orient='index', 
                                                  columns=['title', 
                                                           'artist', 
                                                           'artist_id', 
                                                           'album', 
                                                           'album_id', 
                                                           'release_date',
                                                          'popularity',
                                                          'genre']).reset_index()

In [ ]:
genre_tracks_df

In [ ]:
genre_tracks_df['genre'].value_counts()


In [ ]:
# chill = {}

# chill_results = sp.recommendations(seed_genres=['chill'])
# for track in chill_results['tracks']:
#     id = track['id']
#     chill[id] = [track['name'], # track title
#                         track['artists'][0]['name'], # artist name
#                         track['artists'][0]['id'], # artist id
#                         track['album']['name'], # album title
#                         track['album']['id'], # album id
#                         track['album']['release_date'], # date released 
#                         track['popularity'], # popularity of track
#                         genre] # seed used

# chill_df = pd.DataFrame.from_dict(chill, orient='index', 
#                                                   columns=['title', 
#                                                            'artist', 
#                                                            'artist_id', 
#                                                            'album', 
#                                                            'album_id', 
#                                                            'release_date',
#                                                           'popularity',
#                                                           'genre']).reset_index()

### getting **Audio Features** for tracks:

In [ ]:
# List of tracks used to create audio features dataframe
track_id_list = genre_tracks_df['index'].to_list()

In [ ]:
username = 'MY_USERNAME'
scope = 'user-read-private user-read-email user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

likedtracks_features = []

if token:
    sp = spotipy.Spotify(auth=token)
    for i in range(525):
        begin = i*100
        end = i*100 + 99
        track_master_list = track_id_list[begin:end+1]
        list_features = sp.audio_features(track_master_list)
        likedtracks_features.extend(list_features)

In [ ]:
len(likedtracks_features)

In [ ]:
genre_tracks_features = likedtracks_features

genretracks_features = pd.DataFrame(genre_tracks_features)
genretracks_features.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1, inplace=True)
genretracks_features.head()

In [ ]:
genretracks_info_features = genre_tracks_df.merge(genretracks_features, left_on='index', right_on='id')


In [ ]:
genretracks_info_features.to_csv('data/genretracks_info_features.csv', index=False)

In [ ]:
genretracks_info_features.shape

In [ ]:
genretracks_info_features.drop_duplicates().shape

In [ ]:
sns.boxplot(genretracks_info_features['valence'])

In [ ]:
username = 'MY_USERNAME'
scope = 'user-read-private user-read-email user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

if token:
    sp = spotipy.Spotify(auth=token)
    playlists = sp.current_user_playlists()
    for each in playlists['items']:
        pp.pprint(each['name'])
#     for playlist in playlists['items']:
#         if playlist['owner']['id'] == username:
#             print()
#             print(playlist['name'])
#             print ('  total tracks', playlist['tracks']['total'])
#             results = sp.playlist(playlist['id'],
#                 fields="tracks,next")
#             tracks = results['tracks']
#             show_tracks(tracks)
#             while tracks['next']:
#                 tracks = sp.next(tracks)
#                 show_tracks(tracks)

In [ ]:
playlists

In [ ]:
sp.current_user()